# <center>CITS5508 Lab 2: Bean Classification</center>

**Name:** Chitra M Saraswati<br>
**Student ID:** 21367076<br>

In this workbook we classify beans of seven different varieties. Specifically, we use a Support Vector Classifier and a Stochastic Gradient Descent Classifier; we subsequently compare the performance of these two classification methods.

As per the data description:<br>
<i>"Seven different types of dry beans were used in this research, taking into account the features such as form, shape, type, and structure by the market situation. A computer vision system was developed to distinguish seven different registered varieties of dry beans with similar features in order to obtain uniform seed classification. For the classification model, images of 13,611 grains of 7 different registered dry beans were taken with a high-resolution camera. Bean images obtained by computer vision system were subjected to segmentation and feature extraction stages, and a total of 16 features; 12 dimensions and 4 shape forms, were obtained from the grains."</i>

You can also find the relevant paper below:<br>
KOKLU, M. and OZKAN, I.A., (2020), “Multiclass Classification of Dry Beans Using Computer Vision and Machine Learning Techniques.” Computers and Electronics in Agriculture, 174, 105507. DOI: https://doi.org/10.1016/j.compag.2020.105507

In [ ]:
# Common imports
import os
import numpy as np
import pandas as pd

# To ensure stable output across runs
np.random.seed(35)

# For pretty plots
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
bean_path = os.path.join('DryBeanDataset', 'Dry_Bean_Dataset.xlsx')
os.path.exists(bean_path)

# 1. Setting up

## 1.1 Import data

In [ ]:
from openpyxl import load_workbook
wb = load_workbook(bean_path)

from openpyxl.utils.dataframe import dataframe_to_rows
sheet = wb.active
data = sheet.values
cols = next(data)[0:]
df = pd.DataFrame(data, columns = cols)

In [ ]:
df.head(3)

## 1.2 Exploratory Data Analysis

An explanation of the attributes for this data can be found in the data description. I also thought it was a good idea to go to the paper and have a look at the beans myself. If you refer to the paper, we see that the Bombay and Seker varieties are the most visually distinctive from the other varieties due to their area and roundedness.

As our ML algorithms won't be taking colour into consideration, the Barbunya beans wouldn't be as distinctive to our algorithms as we visually judge them to be.

<img src="bean-varieties.jpg">

<img src="beans-all-together.jpg">

In [ ]:
df.info()

In [ ]:
# Check for any NA values
sample_incomplete_rows = df[df.isnull().any(axis=1)].head()
sample_incomplete_rows

### Histogram of all features

In [ ]:
df.hist(figsize = (20,15), bins="auto")
plt.show()

We discover from the histograms that there are no extreme outliers in our data for any of the attributes. However, some attributes have long tails and are left-skewed (such as that for area, perimeter, major and minor axes length, convex area and diameter). Solidity and ShapeFactor4 are right-skewed.

Additionally, there are some instances of bimodal distributions (such as that for ShapeFactor1) and multimodal distributions (such as that for compactness and ShapeFactor3).

These measures indicate that the data aren't necessarily normally distributed for these features, so we should not assume a normal distribution when selecting the appropriate models/algorithms for our data.

### Scatter plot matrix of most interesting features

In [ ]:
# Creating a scatter plot matrix for the six most interesting attributes:
# Area, eccentricity, equivalent diameter, roundness, compactness, and aspect ratio.

from pandas.plotting import scatter_matrix

bean_attr = ["Area", "Eccentricity", "EquivDiameter", 
             "roundness", "Compactness", "AspectRation"]
scatter_matrix(df[bean_attr], figsize = (12, 8))
plt.show()

The scatter plot matrix above reveal a perfect correlation between the following features:
* area vs equivalent diameter
* eccentricity vs compactness
* eccentricity vs aspect ratio
* compactness vs aspect ratio

This indicates we do not want to attempt classification solely on the above features. This is because classification is best done when distinct groups can be seen in the data (such as that seen in the area vs eccentricity scatterplot, or area vs roundness).

As a result, we should attempt classification on *all* the features provided in the dataset to ensure the best classification methods.

# 2. Viewing class imbalances

In [ ]:
df["Class"].value_counts()

In [ ]:
df["Class"].value_counts().plot(kind='bar')

We see that there is an imbalance between the classes in the Varieties feature. There is a significantly larger amount of the Dermason variety in our sample in comparison to the other varieties (at about 3500 data points). In comparison, the Bombay variety only has about 600 data points.

The issue with class imbalances is non-representative sampling. Ideally, we would have a sample where each bean variety is represented equally.

Going forward, it would be useful to implement ML algorithms which would take into account these significant class imbalances.

# 3. Split to training set and test set

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 35)

In [ ]:
print("Training set:", len(train_set.index), "and testing set:", len(test_set.index))

In [ ]:
# Checking for class imbalances in training set vs test set

fig, (subpl1, subpl2) = plt.subplots(1, 2)
fig.suptitle("Training vs test set")

train_set["Class"].value_counts().plot(kind="bar", ax=subpl1)
test_set["Class"].value_counts().plot(kind="bar", ax=subpl2)

# 4. Data preparation: feature scaling

To prepare our data for our machine learning alogrithms, we'll scale our data using min-max scaling. We'll be using min-max scaling as we don't really have outliers (as per the histograms and scatter plots; refer to EDA in section one). Additionally, SGD is sensitive to feature scaling so we'll stick to using min-max scaling.

In [ ]:
# Now we only use the training set
df = train_set
df.head()

In [ ]:
# Select numeric variables only
df_num = df.select_dtypes(include=[np.number])

In [ ]:
# We use the min-max scaler as there doesn't seem to be extreme outliers 
# (as per histogram and scatter plots; see EDA in section 1).

from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler()

x_train = minmax.fit_transform(df_num)
x_train

In [ ]:
x_train.shape

In [ ]:
# View it in panda's data frame format, just to check
pd_num_tr = pd.DataFrame(x_train)
pd_num_tr.head()

In [ ]:
# Do the same for the testing set
test_num = test_set.select_dtypes(include=[np.number])

x_test = minmax.fit_transform(test_num)
x_test

In [ ]:
x_test.shape

# 5. Using the Support Vector Classifier

In [ ]:
from sklearn.svm import SVC
y_train = train_set["Class"]
y_test = test_set["Class"]

## 5.1 Choose hyperparameters and optimise

We will use GridSearch to find the best hyperparameters for our model. We will play around with the hyperparameters 'kernel', 'gamma', and 'C'.

The code in the following section has been commented out due to processing time.

In [ ]:
# from sklearn.model_selection import GridSearchCV

# param_grid = [
#     {'kernel':['linear', 'rbf', 'sigmoid'], 
#      'gamma':['scale', 'auto'],
#      'C':[1,2,3]}
# ]

# svc = SVC()
# grid_search = GridSearchCV(svc, param_grid, cv=5, return_train_score=True)
# grid_search.fit(x_train, y_train)

In [ ]:
# grid_search.best_params_

In [ ]:
# grid_search.best_estimator_

In [ ]:
# cvres = grid_search.cv_results_
# pd.DataFrame(grid_search.cv_results_)

The best hyperparameters found using GridSearch were: <br>
* C: 3
* Gamma: scale (default)
* Kernel: rbf (default)

So we will specify these for our model.

## 5.2 Fit the training model

In [ ]:
svc = SVC(kernel="rbf", C=3)
svc.fit(x_train, y_train)

In [ ]:
y_pred_svc = svc.predict(x_test)

In [ ]:
svc.classes_

## 5.3 Cross-validate

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, plot_confusion_matrix

cm_svc = confusion_matrix(y_test, y_pred_svc)
cm_svc

In [ ]:
# Confusion matrix for correct classification
# plt.matshow(cm_svc, cmap = plt.cm.gray)
# plt.show()

In [ ]:
# And more specifically:
plot_confusion_matrix(svc, x_test, y_test)

The confusion matrix here looks quite good as most values are on the main diagonal, i.e. they were classified correctly (the darker the square, the less values there were classified in that category).

The Bombay variety (class '1' in the plot) looks darker than the other varieties; indeed, there were fewer Bombay beans than there were other varieties (refer to the section on EDA in Part 1). We could also check if the classifier does not perform as well on the Bombay than for the other varieties. This is done below.

In [ ]:
# Focusing on plotting the errors
row_sums = cm_svc.sum(axis=1, keepdims=True)
ncm_svc = cm_svc / row_sums
np.fill_diagonal(ncm_svc, 0)
plt.matshow(ncm_svc, cmap=plt.cm.gray)
plt.show()

The rows and columns for the Bombay bean (class 1) is dark, indicating that Bombay beans tend to get classified correctly. So we do not need to worry about the classifier not having performed as well on Bombay than for the other varieties.

After Bombay beans come the Seker beans (class 5); they also tend to get classified correctly, even though not to the same extent.

Demarson and Sira varieties (classes 3 and 6 respectively) tend to get confused for each other, in both directions–moreso Sira beans being misclassified as a Demarson variety than the other way around. This confusion for two varieties also occur for the  Barbunya (class 0) and Cali (class 2) varieties.

The column for Sira beans (class 6) is quite bright meaning that other varieties tend to be misclassified as Sira beans. 

# 6. Using the Stochastic Gradient Descent Classifier

Comment to self: This is basically a linear regression model.. but slower.

Also–it's called 'stochastic' because we randomly sample from the training set and calculate the gradient only for that instance, rather than calculating the gradient at every instance. This method is especially useful for large data sets.

In [ ]:
from sklearn.linear_model import SGDClassifier

y_train = train_set["Class"]
y_test = test_set["Class"]
y_train

## 6.1 Choose hyperparameters and optimise
The hyperparameters we will consider playing around with for this classification method are loss and penalty. I didn't consider trying to change the other hyperparameters as we don't have problems with outliers or any strange data in our dataset.

In [ ]:
# from sklearn.model_selection import GridSearchCV

# param_grid = [
#     {'loss':['hinge', 'log', 'squared_hinge'], # We don't have a lot of outliers so modified_huber not checked
#      'penalty':['l2', 'l1', 'elasticnet']}
# ]

# sgdc = SGDClassifier()
# grid_search = GridSearchCV(sgdc, param_grid, cv=5, return_train_score=True)
# grid_search.fit(x_train, y_train)

In [ ]:
# grid_search.best_params_

In [ ]:
# grid_search.best_estimator_

In [ ]:
# cvres = grid_search.cv_results_
# pd.DataFrame(grid_search.cv_results_)

We find that we want the loss parameter to be 'hinge' and penalty hyperparameter to be 'l1'.

## 6.2 Train the SGD Classifier

In [ ]:
sgdc = SGDClassifier(random_state = 35, loss = 'hinge', penalty='l1')
sgdc.fit(x_train, y_train)

In [ ]:
score = sgdc.score(x_train, y_train)
print("Training score: ", score) 

In [ ]:
y_pred_sgdc = sgdc.predict(x_test)

## 6.3 Cross-validate

In [ ]:
cm_sgdc = confusion_matrix(y_test, y_pred_sgdc)
print(cm_sgdc)

In [ ]:
# plt.matshow(cm_sgdc, cmap = plt.cm.gray)
# plt.show()

In [ ]:
plot_confusion_matrix(sgdc, x_test, y_test)

In this instance the confusion matrix looks a bit worse. Most of the brightness is in the diagonal but there is a high number of errors where Sira beans are misclassified as Seker beans, which is concerning.

In [ ]:
row_sums = cm_sgdc.sum(axis=1, keepdims=True)
ncm_sgdc = cm_sgdc / row_sums
np.fill_diagonal(ncm_sgdc, 0)
plt.matshow(ncm_sgdc, cmap=plt.cm.gray)
plt.show()

We observe that with this classification method, the most misclassification occured when Seker beans (class 5) are misclassified as Sira (class 6).

In [ ]:
#QN-ME: what does cv=3 below indicate for cross_val_score?
# It is how many k-folds; in this instance, 3. Default is 5-fold
from sklearn.model_selection import cross_val_score
cross_val_score(sgdc, x_train, y_train, cv=3, scoring="accuracy")

# 7. Comparing the two classifiers

We will now compare the two classifiers' confusion matrices side by side. The confusion matrices on the left are that for the SVC whilst the one on the right is for the SGDC. 

In [ ]:
fig, (subpl1, subpl2) = plt.subplots(1, 2)
fig.suptitle("Confusion Matrices")

subpl1.matshow(cm_svc, cmap = plt.cm.gray)
subpl2.matshow(cm_sgdc, cmap = plt.cm.gray)

In [ ]:
fig, (subpl1, subpl2) = plt.subplots(1, 2)
fig.suptitle("Confusion Matrices: Errors")

subpl1.matshow(ncm_svc, cmap=plt.cm.gray)
subpl2.matshow(ncm_sgdc, cmap=plt.cm.gray)

Ultimately, the SVC seems to be a better classifier for our data as there were less serious misclassifications than in the SGDC.